In [1]:
workspace_dir = '.'

In [2]:
#https://drive.google.com/file/d/1CetKZ0gdM5W6q_AYxrVS52yVsNLcb2PM/view?usp=drive_link
!gdown --id '1CetKZ0gdM5W6q_AYxrVS52yVsNLcb2PM' \
    --output "{workspace_dir}/specs.zip"
#!gdown --id '185JzJk6_0vF99A3qHTaEQPsUwklRKH1b' \
#    --output "{workspace_dir}/specs_100.zip"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CetKZ0gdM5W6q_AYxrVS52yVsNLcb2PM
To: /content/specs.zip
100% 3.60G/3.60G [00:54<00:00, 65.5MB/s]


In [3]:
!unzip "{workspace_dir}/specs"
#!unzip "{workspace_dir}/specs_100.zip"

Streaming output truncated to the last 5000 lines.
  inflating: specs/889_215.png       
  inflating: specs/889_216.png       
  inflating: specs/889_217.png       
  inflating: specs/889_218.png       
  inflating: specs/889_219.png       
  inflating: specs/889_220.png       
  inflating: specs/889_221.png       
  inflating: specs/889_222.png       
  inflating: specs/889_223.png       
  inflating: specs/889_224.png       
  inflating: specs/889_225.png       
  inflating: specs/889_226.png       
  inflating: specs/889_227.png       
  inflating: specs/889_228.png       
  inflating: specs/889_229.png       
  inflating: specs/889_230.png       
  inflating: specs/889_231.png       
  inflating: specs/889_232.png       
  inflating: specs/889_233.png       
  inflating: specs/889_234.png       
  inflating: specs/889_235.png       
  inflating: specs/889_236.png       
  inflating: specs/889_237.png       
  inflating: specs/889_238.png       
  inflating: specs/889_239.png       

In [4]:
!pip3 install torchmetrics einops scikit_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00


In [5]:
import os
import random
import timeit
from functools import reduce

from skimage import io
from argparse import Namespace
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchmetrics.classification import MulticlassConfusionMatrix as ConfusionMatrix
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import einops
from einops.layers.torch import Rearrange


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
dataset_list = 'dataset_list.txt'
spectrograms = 'specs'
#spectrograms = 'specs_100'

wav = Namespace(
    n_fft=2048,
    hop_length=512,
    n_mels=40
)

mofig= Namespace(
    random_seed = 1111,
    batch_size = 16,
    epochs = 40,
    learning_rate = 3e-3,
    num_classes = 932,
    patch_height = 256, #100,
    patch_width = 1,
    img_size = 256, #100,
    in_channels = 4,
    num_heads = 8,
    dropout = 0.1,
    hidden_dim = 1536, #768,
    adam_weight_decay = 0,
    adam_betas = (0.9, 0.999),
    num_encoders = 8,
    emb_dim = 768, #128,
    num_patches = (256 ** 2) // (256 * 1), #(100 ** 2) // (100 * 1),
    confusion_mat = 'confusion_matrix.png',
    checkpoint = 'checkpoint.pt'
)

In [7]:
random.seed(mofig.random_seed)
torch.manual_seed(mofig.random_seed)
torch.cuda.manual_seed(mofig.random_seed)
torch.cuda.manual_seed_all(mofig.random_seed)
torch.backends.cudnn.deterministic = True

In [17]:
with open(os.path.join(workspace_dir, 'dataset_list.txt'), 'w+') as f:
    for root, dirs, files in os.walk(spectrograms):
        for idx in range(len(files)):
            comma = ','
            if idx == 0:
              comma = ''
            if '.png' in file:
              f.write(comma + os.path.join(root, files[idx]))
f.close()

In [18]:
f = open(os.path.join(workspace_dir, 'dataset_list.txt'), 'r').read()
f_read = f.split(',')
for idx, entry in enumerate(f_read):
  if '.png' not in entry:
    print(idx, entry)

In [9]:
class Embedding(nn.Module):
    def __init__(self,
                 patch_height=mofig.patch_height,
                 patch_width=mofig.patch_width,
                 emb_dim=mofig.emb_dim,
                 latent_size=mofig.hidden_dim):
        super(Embedding, self).__init__()
        self.patch_height = patch_height
        self.patch_width = patch_width
        self.emb_dim = emb_dim
        self.input_size = self.patch_height * self.patch_width * 4
        self.latent_size = latent_size
        self.batch_size = mofig.batch_size

        # Patchify
        self.patches = Rearrange('b c (ph h) (pw w) -> b (h w) (ph pw c)',
                                 ph=self.patch_height,
                                 pw=self.patch_width)

        # Linear Projection
        self.projection = nn.Linear(self.input_size, self.latent_size)

        # Class Token
        self.class_token = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(device)

        # Position Embedding
        self.pos_emb = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(device)

    def forward(self, x):
        x = self.patches(x)
        x = self.projection(x)
        b, n, _ = x.shape
        x = torch.cat((self.class_token, x), dim=1)
        pos_emb = einops.repeat(self.pos_emb, 'b 1 d -> b m d', m=n+1)
        x += pos_emb
        return x

class EncoderBlock(nn.Module):
    def __init__(self,
                 latent_size=mofig.hidden_dim,
                 num_heads=mofig.num_heads,
                 dropout=mofig.dropout):
        super(EncoderBlock, self).__init__()
        self.latent_size = latent_size
        self.num_heads = num_heads
        self.dropout = dropout

        self.norm = nn.LayerNorm(self.latent_size)

        self.attention = nn.MultiheadAttention(self.latent_size,
                                               self.num_heads,
                                               self.dropout)
        self.ff = nn.Sequential(
                    nn.Linear(self.latent_size, self.latent_size*3),
                    nn.GELU(),
                    nn.Dropout(self.dropout),
                    nn.Linear(self.latent_size*3, self.latent_size),
                    nn.Dropout(self.dropout)
                    )

    def forward(self, x):
        x_residual = x
        x = self.norm(x.clone())
        x = self.attention(x, x, x)[0]
        x_residual += x
        x = self.norm(x_residual.clone())
        x = self.ff(x)
        return x + x_residual


class ViT(nn.Module):
    def __init__(self,
                 num_encoders=mofig.num_encoders,
                 latent_size=mofig.hidden_dim,
                 dropout=mofig.dropout,
                 num_classes=mofig.num_classes):
        super(ViT, self).__init__()
        self.num_encoders = num_encoders
        self.latent_size = latent_size
        self.dropout = dropout
        self.num_classes = num_classes

        self.embedding = Embedding()

        self.encoder = nn.ModuleList([EncoderBlock() for i in range(self.num_encoders)])

        self.ff = nn.Sequential(
                    nn.LayerNorm(self.latent_size),
                    nn.Linear(self.latent_size, self.latent_size),
                    nn.Linear(self.latent_size, self.num_classes)
                    )

    def forward(self, x):
        x = self.embedding(x)

        for e in self.encoder:
            x = e.forward(x)

        class_token_emb = x[:, 0]

        return self.ff(class_token_emb)



In [10]:
class SRData(Dataset):
    def __init__(self, dataset_list):
        self.dataset_list = dataset_list
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.dataset_list)

    def __getitem__(self, idx):
        img = io.imread(self.dataset_list[idx])
        img = self.transform(img)
        img = torch.div(img, mofig.img_size - 1)
        return img, self.getLabel(idx)

    def getLabel(self, idx):
        file = self.dataset_list[idx]
        filename = file.split('/')[-1]
        label = int(filename.split('_')[0])
        return label

In [11]:
def getParameterCount(model):
    return sum(p.numel() for p in model.parameters())


def getROCScore(label, pred):
    prob_estimate = F.softmax(pred, dim=1).detach()
    return roc_auc_score(label, prob_estimate, multi_class="ovr")


def getEER(labels, pred):
    CM = ConfusionMatrix(mofig.num_classes).to(device)
    confusion_mat = torch.Tensor(CM(pred, labels)).to(device)
    #print(confusion_mat)
    EER = []
    for n in range(mofig.num_classes):
        true_pos = confusion_mat[n][n]
        false_pos = torch.sum(torch.Tensor([i[n] for i in confusion_mat])).to(device) - true_pos
        false_neg = torch.sum(torch.Tensor(confusion_mat[n])).to(device) - true_pos
        true_neg = torch.sum(torch.Tensor([sum(torch.Tensor(i)) for i in confusion_mat])).to(device) - true_pos - false_pos - false_neg

        FAR = false_pos / (false_pos + true_neg + 1) * 100
        FRR = false_neg / (false_neg + true_pos + 1) * 100

        EER.append((FAR + FRR) / 2)

    return EER, confusion_mat


def computAVG(matrices):
    count = len(matrices)
    i_len = len(len(matrices[0]))
    j_len = len(len(matrices[0][0]))
    avg_matrix = np.zeros((i,j))
    for i in range(i_len):
        for j in range(j_len):
            pos_sum = 0
            for matrix in matrices:
                pos_sum += matrix[i][j]
            avg_matrix[i][j] = pos_sum / count
    return avg_matrix


def saveConfusionMatrixPlot(matrix, dest):
    plt.imshow(matrix, cmap='viridis')
    plt.colorbar()
    plt.title('Confusion Matrix')
    classes = ['classes' + i for i in range(mofig.num_classes)]
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            plt.text(j, i, f"{matrix[i, j]}", ha='center', va='center', color='black')

    plt.savefig(dest, format='png')

In [21]:
dataset_list = open(dataset_list).read()
dataset_list = dataset_list.split(',')
dataset = SRData(dataset_list)
dataset_len = len(dataset_list)
train_split = int(dataset_len * 0.8)
val_split = int(dataset_len * 0.1)
test_split = dataset_len - train_split - val_split

train_set, val_set, test_set = random_split(dataset,
                                                      [train_split, val_split, test_split])
train_loader = DataLoader(train_set, batch_size=mofig.batch_size, shuffle=True)
val_loader = DataLoader(train_set, batch_size=mofig.batch_size, shuffle=True)
test_loader = DataLoader(train_set, batch_size=mofig.batch_size, shuffle=True)

In [13]:
model = ViT().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),
                              betas=mofig.adam_betas,
                              lr=mofig.learning_rate,
                              weight_decay=mofig.adam_weight_decay)

In [ ]:
torch.autograd.set_detect_anomaly(True)
start = timeit.default_timer()

for epoch in tqdm(range(mofig.epochs)):
    model.train()
    train_labels =[]
    train_preds = []
    train_roc_scores = []
    train_EERs = []
    train_running_loss = 0
    for idx, data in enumerate(tqdm(train_loader)):
        img = data[0].float().to(device)
        label = data[1].to(device)

        output = model(img)
        pred = torch.argmax(output, dim=1)

        train_labels.extend(label.cpu().detach())
        train_preds.extend(pred.cpu().detach())

        #print(f'Output shape: {output.shape}')
        #print(f'Label shape: {label.shape}')

        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_running_loss += loss.item()

        #train_roc_score = getROCScore(output, label)
        #train_roc_scores.extend(train_roc_score.cpu().detach())

        #train_EER, _ = getEER(label, pred)
        #train_EERs.extend(train_EER)

        print(f"Train Accuracy: "
            f"{sum(1 for x,y in zip(train_preds, train_labels) if x == y) / len(train_labels):.4f}")

    if epoch == 5:
        torch.save_to_dict(model, mofig.checkpoint)

    #train_loss = train_running_loss / (idx + 1)

    model.eval()
    val_labels = []
    val_pred = []
    val_roc_score = []
    val_EERs = []
    val_running_loss = 0
    with torch.no_grad():
        for idx, (img, label) in enumerate(tqdm(val_loader)):
            img = img.float().to(device)
            label = label.to(device)

            output = model(img)
            pred = torch.argmax(output, dim=1)

            val_labels.extend(label.cpu().detach())
            val_pred.extend(pred.cpu().detach())

            loss = criterion(output, label)
            val_running_loss += loss.item()

            #val_roc_score = getROCScore(output, label)
            #val_roc_scores.extend(val_roc_score.cpu().detach())

            #val_EER, _ = getEER(label, pred)
            #val_EERs.extend(val_EER)

    val_loss = val_running_loss / (idx + 1)
    print("-"*30)
    print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
    print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
    print(f"Train Accuracy EPOCH {epoch+1}: "
            f"{sum(1 for x,y in zip(train_preds, train_labels) if x == y) / len(train_labels):.4f}")
    print(f"Valid Accuracy EPOCH {epoch+1}: "
            f"{sum(1 for x,y in zip(val_preds, val_labels) if x == y) / len(val_labels):.4f}")
    #print(f"Train ROC Score {epoch+1}: {sum(train_roc_scores) / len(train_roc_scores)}")
    #print(f"Valid ROC Score {epoch+1}: {sum(val_roc_scores) / len(val_roc_scores)}")
    #print(f"EER Score: {val_sum(EERs) / len(val_EERs)}")
    print("-"*30)

stop = timeit.default_timer()
print(f'Training time: {stop - start: .2f}sec')

  0%|          | 1/5699 [00:00<45:49,  2.07it/s]

Train Accuracy: 0.0000



  0%|          | 2/5699 [00:00<42:36,  2.23it/s]

Train Accuracy: 0.0000



  0%|          | 3/5699 [00:01<41:22,  2.29it/s]

...


  1%|          | 52/5699 [00:22<39:48,  2.36it/s]

Train Accuracy: 0.0012



  1%|          | 53/5699 [00:22<39:44,  2.37it/s]

Train Accuracy: 0.0012



  1%|          | 54/5699 [00:22<39:51,  2.36it/s]

In [ ]:
print(f'Model parameter count: {getParameterCount(model)}')

In [ ]:
preds = []
labels = []
roc_scores = []
EERs = []
CM = []
model.eval()
with torch.no_grad():
    for idx, data in enumerate(tqdm(test_loader)):
        img = data[0].float().to(device)
        label = data[1].to(device)

        output = model(img)

        labels.extend(label.detach())
        preds.extend([int(i) for i in torch.argmax(output, dim=1)])
        #roc_scores.extend(getROCScore(output, label).cpu().detach())
        EER, confusion_mat = getEER(label, output)
        EERs.extend(EER)
        CM.append(confusion_mat)

print("-"*30)
print(f"Accuracy: {sum(1 for x, y in zip(preds, labels) if x == y) / len(labels):.4f}")
#print(f"ROC Score: {sum(roc_scores) / len(roc_scores)}")
print(f"EER Score: {sum(EERs) / len(EERs)}")
print("-"*30)

avg_conf_mat = computAVG(EERS)
avg_conf_mat = reduce("+", EERS) / len(EERS)
saveConfusionMatrixPlot(avg_conf_mat, mofig.confusion_mat)